In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [3]:
from proj1_helpers import *
test=load_csv_data("../dataset/test.csv", sub_sample=False)
train=load_csv_data("../dataset/train.csv", sub_sample=False)

In [5]:
train

(array([ 1., -1., -1., ...,  1., -1., -1.]),
 array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
        [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
        [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
        ..., 
        [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
        [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
        [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]]),
 array([100000, 100001, 100002, ..., 349997, 349998, 349999]))

In [7]:
from least_squares import *
mse,weights=least_squares(train[0],train[1])

In [8]:
mse

0.3396868094770345